In [ ]:
import yfinance as yf
import pandas as pd
from datetime import date
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [ ]:
# top 10 holdings from VanEck Retail ETF
holdings = ['AMZN', 'COST', 'WMT', 'HD', 'LOW', 'TJX', 'ORLY', 'MCK', 'CVS', 'TGT']

In [ ]:
# use data for training and testing the model (80/20)
START = "2000-01-01"
TODAY = date.today().strftime("%Y-%m-%d")
stock_data = []
for ticker in holdings:
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    data['Ticker'] = ticker
    stock_data.append(data)
stock_df = pd.concat(stock_data, axis=0, ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
stock_df

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2000-01-03,4.075000,4.478125,3.952344,4.468750,4.468750,322352000,AMZN
1,2000-01-04,4.268750,4.575000,4.087500,4.096875,4.096875,349748000,AMZN
2,2000-01-05,3.525000,3.756250,3.400000,3.487500,3.487500,769148000,AMZN
3,2000-01-06,3.565625,3.634375,3.200000,3.278125,3.278125,375040000,AMZN
4,2000-01-07,3.350000,3.525000,3.309375,3.478125,3.478125,210108000,AMZN
...,...,...,...,...,...,...,...,...
62235,2024-09-23,154.770004,155.740005,152.910004,155.289993,155.289993,2927700,TGT
62236,2024-09-24,155.899994,157.830002,155.500000,156.179993,156.179993,3114000,TGT
62237,2024-09-25,156.289993,156.919998,154.160004,155.070007,155.070007,2319900,TGT
62238,2024-09-26,156.529999,157.740005,155.119995,155.979996,155.979996,2757900,TGT


In [ ]:
# run a random forest on each ticker using 80% of data for training and 20% of data for testing
def train(df,ticker):
    ticker_df = df[df['Ticker'] == ticker]
    X = ticker_df[['Open', 'High', 'Low', 'Volume']]
    y = ticker_df['Adj Close']  # adjusted closing price
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    rf = RandomForestRegressor(n_estimators=100,
                               max_depth = 10,
                               random_state=42)
    rf.fit(X_train, y_train)
    model = rf.predict(X_test)
    return y_test, model

In [ ]:
demo_test, demo_model = train(stock_df, holdings[5])